# RTM Simulation: Flat Small-World Network (MFPT)

This notebook demonstrates the **small-world network regime** in the RTM framework.

## Theoretical Background

### Watts-Strogatz Small-World Model

The Watts-Strogatz model creates networks that interpolate between:
- **Regular lattice** (p=0): high clustering, long average paths
- **Random graph** (p=1): low clustering, short average paths
- **Small-world** (0 < p < 1): high clustering AND short average paths

### RTM Predictions

For flat small-world networks, RTM predicts:
- The MFPT shows **logarithmic** scaling with N: $T \propto \log(N)$
- When using $L = \sqrt{N}$ as scale, a power-law fit yields $\alpha \approx 2.1$

**Expected result:** $\alpha \approx 2.0-2.1$

## 1. Setup and Imports

In [ ]:
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import pandas as pd

RANDOM_SEED = 42
rng = np.random.default_rng(RANDOM_SEED)

print("Libraries loaded successfully!")

## 2. Watts-Strogatz Network Generation

In [ ]:
def create_watts_strogatz_graph(n, k, p, rng):
    """
    Create a Watts-Strogatz small-world graph.
    
    Parameters:
    - n: number of nodes
    - k: each node connected to k nearest neighbors
    - p: rewiring probability
    """
    if k % 2 != 0:
        k = k - 1
    
    adj = {i: set() for i in range(n)}
    
    # Create ring lattice
    for i in range(n):
        for j in range(1, k // 2 + 1):
            neighbor_right = (i + j) % n
            neighbor_left = (i - j) % n
            adj[i].add(neighbor_right)
            adj[neighbor_right].add(i)
            adj[i].add(neighbor_left)
            adj[neighbor_left].add(i)
    
    # Rewiring
    for i in range(n):
        for j in range(1, k // 2 + 1):
            neighbor = (i + j) % n
            if rng.random() < p and neighbor in adj[i]:
                possible_targets = [x for x in range(n) if x != i and x not in adj[i]]
                if len(possible_targets) > 0:
                    new_target = rng.choice(possible_targets)
                    adj[i].discard(neighbor)
                    adj[neighbor].discard(i)
                    adj[i].add(new_target)
                    adj[new_target].add(i)
    
    return {i: list(neighbors) for i, neighbors in adj.items()}

print("Network generation function defined.")

## 3. Visualize a Small-World Network

In [ ]:
# Create a small example network
n_example = 30
k_example = 4
p_example = 0.1

adj_example = create_watts_strogatz_graph(n_example, k_example, p_example, rng)

# Visualize as a ring with edges
fig, ax = plt.subplots(figsize=(8, 8))

# Position nodes in a circle
angles = np.linspace(0, 2*np.pi, n_example, endpoint=False)
x = np.cos(angles)
y = np.sin(angles)

# Draw edges
for i, neighbors in adj_example.items():
    for j in neighbors:
        if i < j:  # Avoid drawing twice
            # Regular edge (short) vs rewired edge (long)
            dist = min(abs(i-j), n_example - abs(i-j))
            color = 'red' if dist > k_example//2 + 1 else 'lightgray'
            alpha = 0.8 if dist > k_example//2 + 1 else 0.3
            ax.plot([x[i], x[j]], [y[i], y[j]], color=color, alpha=alpha, linewidth=1)

# Draw nodes
ax.scatter(x, y, s=100, c='blue', zorder=5)
for i in range(n_example):
    ax.annotate(str(i), (x[i], y[i]), fontsize=8, ha='center', va='center', color='white')

ax.set_title(f'Watts-Strogatz Network (N={n_example}, k={k_example}, p={p_example})\nRed = rewired shortcuts', fontsize=12)
ax.set_aspect('equal')
ax.axis('off')
plt.tight_layout()
plt.show()

## 4. Random Walk and MFPT Functions

In [ ]:
def random_walk_mfpt(adj, source, target, max_steps, rng):
    """Perform random walk and return first-passage time."""
    current = source
    steps = 0
    
    while steps < max_steps:
        if current == target:
            return steps
        neighbors = adj[current]
        if len(neighbors) == 0:
            return -1
        current = rng.choice(neighbors)
        steps += 1
    
    return -1

print("Random walk function defined.")

## 5. Run Simulation

In [ ]:
# Parameters
NETWORK_SIZES = np.array([100, 200, 400, 800, 1600, 3200])
K = 6
P_REWIRE = 0.1
N_REALIZATIONS = 5
N_PAIRS = 50
N_WALKS = 10
MAX_STEPS = 500_000

rng = np.random.default_rng(RANDOM_SEED)
results = []

print("Running simulations...\n")
print(f"{'N':>6} | {'L=√N':>8} | {'MFPT':>12}")
print("-" * 35)

for N in NETWORK_SIZES:
    L = np.sqrt(N)
    mfpt_values = []
    
    for _ in range(N_REALIZATIONS):
        adj = create_watts_strogatz_graph(N, K, P_REWIRE, rng)
        
        for _ in range(N_PAIRS):
            source = rng.integers(0, N)
            target = rng.integers(0, N)
            while target == source:
                target = rng.integers(0, N)
            
            pair_times = []
            for _ in range(N_WALKS):
                fpt = random_walk_mfpt(adj, source, target, MAX_STEPS, rng)
                if fpt >= 0:
                    pair_times.append(fpt)
            
            if pair_times:
                mfpt_values.append(np.mean(pair_times))
    
    T_mean = np.mean(mfpt_values)
    T_std = np.std(mfpt_values)
    T_sem = T_std / np.sqrt(len(mfpt_values))
    
    results.append({'N': N, 'L': L, 'T_mean': T_mean, 'T_std': T_std, 'T_sem': T_sem})
    print(f"{N:>6} | {L:>8.2f} | {T_mean:>12.2f}")

df = pd.DataFrame(results)
print("\nSimulation complete!")

## 6. Fit Power Law: $T = T_0 \cdot L^\alpha$

In [ ]:
log_L = np.log10(df['L'].values)
log_T = np.log10(df['T_mean'].values)

slope, intercept, r_value, p_value, std_err = stats.linregress(log_L, log_T)

alpha = slope
T0 = 10**intercept
R_squared = r_value**2

print("=" * 50)
print("POWER LAW FIT RESULTS")
print("=" * 50)
print(f"\nFitted exponent: α = {alpha:.4f} ± {std_err:.4f}")
print(f"R² = {R_squared:.6f}")
print(f"\nExpected (RTM): α ≈ 2.1")
print(f"Deviation: {abs(alpha - 2.1) / 2.1 * 100:.1f}%")

## 7. Visualization

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Log-log scaling
ax1 = axes[0]
ax1.errorbar(df['L'], df['T_mean'], yerr=df['T_sem'],
             fmt='o', markersize=10, capsize=4, color='blue',
             label='Simulation data')

L_fit = np.linspace(df['L'].min(), df['L'].max(), 100)
T_fit = T0 * L_fit**alpha

ax1.plot(L_fit, T_fit, 'r-', linewidth=2,
         label=f'Fit: α = {alpha:.3f}')

ax1.set_xscale('log')
ax1.set_yscale('log')
ax1.set_xlabel('Effective Length L = √N', fontsize=12)
ax1.set_ylabel('Mean First-Passage Time T', fontsize=12)
ax1.set_title(f'Small-World Network: T ∝ L^α (R² = {R_squared:.4f})', fontsize=14)
ax1.legend(fontsize=11)
ax1.grid(True, alpha=0.3, which='both')

# Plot 2: Linear scale
ax2 = axes[1]
ax2.errorbar(df['N'], df['T_mean'], yerr=df['T_sem'],
             fmt='s', markersize=10, capsize=4, color='green',
             label='Simulation data')
ax2.set_xlabel('Network Size N', fontsize=12)
ax2.set_ylabel('Mean First-Passage Time T', fontsize=12)
ax2.set_title('MFPT vs Network Size', fontsize=14)
ax2.legend(fontsize=11)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 8. Summary

In [ ]:
print("=" * 60)
print("SUMMARY: Flat Small-World Network Simulation")
print("=" * 60)
print(f"""
RTM Prediction for Small-World Networks: α ≈ 2.1

This Simulation:
  • Fitted exponent: α = {alpha:.4f} ± {std_err:.4f}
  • R² = {R_squared:.6f}
  • Deviation from expected: {abs(alpha - 2.1) / 2.1 * 100:.1f}%

INTERPRETATION:
Small-world networks have "shortcuts" that allow rapid traversal.
The MFPT scaling with α ≈ 2 falls between:
  • Ballistic (α ≈ 1): direct paths
  • Diffusive (α ≈ 2): random exploration

The small-world topology accelerates transport compared to regular
lattices, but random walks still exhibit near-diffusive behavior
because the walker doesn't "know" which shortcuts to take.

STATUS: {'✓ CONFIRMED' if abs(alpha - 2.1) < 0.2 else '⚠ NEEDS REVIEW'}
""")

## 9. Save Results

In [ ]:
df.to_csv('output/flat_small_world_data.csv', index=False)
print("Data saved to output/flat_small_world_data.csv")
df